## Import Libraries

In [ ]:
import FileManager
import WordCleaner
import Indexer
%load_ext autoreload
%autoreload 2

## Load Files

In [ ]:
dataset = FileManager.csvToDict('testing.csv')

## Stemming

In [ ]:
stemmed_dataset = {}
# To clear the contents of the file
stemmed_file_writer, file = FileManager.openCSVWriter('stemmed.csv',['id','text'])
for row in dataset:
    stemmed_dataset[row] = WordCleaner.stem(dataset[row], 'porter')
    stemmed_file_writer.writerow({'id': row, 'text': stemmed_dataset[row]})

### Removing stop words

In [ ]:
stemmed_filtered_dataset = {}
stemmed_file_writer, file = FileManager.openCSVWriter('stemmed.csv',['id','text'])
for key in stemmed_dataset:
    stemmed_filtered_dataset[key] = WordCleaner.removeStopWords(stemmed_dataset[key])
    stemmed_file_writer.writerow({'id': key, 'text': stemmed_filtered_dataset[key]})

## OR Lemmatization

In [ ]:
lemmatized_dataset = {}
lemmatized_file_writer, file = FileManager.openCSVWriter('lemmatized.csv',['id','text'])
for row in dataset:
    lemmatized_dataset[row] = WordCleaner.lemmatize(dataset[row])
    lemmatized_file_writer.writerow({'id': row, 'text': lemmatized_dataset[row]})

### Removing Stopwords

In [ ]:
lemmatized_filtered_dataset = {}
lemmatized_file_writer, file = FileManager.openCSVWriter('lemmatized.csv',['id','text'])
for key in lemmatized_dataset:
    lemmatized_filtered_dataset[key] = WordCleaner.removeStopWords(lemmatized_dataset[key])
    lemmatized_file_writer.writerow({'id': key, 'text': lemmatized_filtered_dataset[key]})

## Creating the inverted index
### for stemmed

In [ ]:
stemmed_inverted_index = Indexer.getInvertedIndex(stemmed_filtered_dataset)
stemmed_inverted_index

### for lemmatized

In [ ]:
lemmatized_inverted_index = Indexer.getInvertedIndex(lemmatized_filtered_dataset)
lemmatized_inverted_index

## Calculating tf-idf for the document
### for stemmed

In [ ]:
(tfidf_matrix, df) = Indexer.calculateTF_IDF(stemmed_filtered_dataset)
df

### for lemmatized

In [ ]:
(tfidf_matrix, df) = Indexer.calculateTF_IDF(lemmatized_filtered_dataset)
df